In [73]:
import numpy as np
import pandas as pd
import os

In [74]:
from sklearn.preprocessing import OneHotEncoder, LabelEncoder
from sklearn.ensemble import RandomForestRegressor

path = ""
train = pd.read_csv(path + "train.csv")
test = pd.read_csv(path + "test.csv")

# One-hot-encode categorical variables
train['dataset'] = "train"
test['dataset'] = "test"
data = pd.concat([train,test], axis = 0)
categorical = ['property_type','room_type','bed_type','cancellation_policy','city']
data = pd.get_dummies(data, columns = categorical)



# Select only numeric data and impute missing values as 0
numerics = ['uint8','int16', 'int32', 'int64', 'float16', 'float32', 'float64']
train_x = data[data.dataset == "train"] \
    .select_dtypes(include=numerics) \
    .drop("log_price", axis = 1) \
    .fillna(0) \
    .values

test_x = data[data.dataset == "test"] \
    .select_dtypes(include=numerics) \
    .drop("log_price", axis = 1) \
    .fillna(0) \
    .values
    
train_y = data[data.dataset == "train"].log_price.values

# Train a Random Forest model with cross-validation

from sklearn.model_selection import KFold
cv_groups = KFold(n_splits=3)
regr = RandomForestRegressor(random_state = 0, n_estimators = 10)

for train_index, test_index in cv_groups.split(train_x):
    
    # Train the model using the training sets
    regr.fit(train_x[train_index], train_y[train_index])
    
    # Make predictions using the testing set
    pred_rf = regr.predict(train_x[test_index])
    
    # Calculate RMSE for current cross-validation split
    rmse = str(np.sqrt(np.mean((train_y[test_index] - pred_rf)**2)))
    
    print("RMSE for current split: " + rmse)
    
# Create submission file
regr.fit(train_x, train_y)
final_prediction = regr.predict(test_x)

submission = pd.DataFrame(np.column_stack([test.id, final_prediction]), columns = ['id','log_price'])
submission.to_csv("sample_submission.csv", index = False)


RMSE for current split: 0.423195134824
RMSE for current split: 0.416264460802
RMSE for current split: 0.427645343345


In [75]:
print(data)

       accommodates                                          amenities  \
0                 3  {"Wireless Internet","Air conditioning",Kitche...   
1                 7  {"Wireless Internet","Air conditioning",Kitche...   
2                 5  {TV,"Cable TV","Wireless Internet","Air condit...   
3                 4  {TV,"Cable TV",Internet,"Wireless Internet",Ki...   
4                 2  {TV,Internet,"Wireless Internet","Air conditio...   
5                 2  {TV,"Wireless Internet",Heating,"Smoke detecto...   
6                 3  {TV,Internet,"Wireless Internet","Air conditio...   
7                 2  {TV,"Cable TV","Wireless Internet","Wheelchair...   
8                 2  {TV,"Cable TV","Wireless Internet","Pets live ...   
9                 2  {"Wireless Internet","Air conditioning",Kitche...   
10                2  {Internet,"Wireless Internet","Air conditionin...   
11                4  {TV,"Cable TV",Internet,"Wireless Internet",Ki...   
12                2  {TV,Internet,"Wir

In [76]:
type(train["amenities"][0])

str

In [77]:
x = train["amenities"][0]
print(x)
print(type(x))

{"Wireless Internet","Air conditioning",Kitchen,Heating,"Family/kid friendly",Essentials,"Hair dryer",Iron,"translation missing: en.hosting_amenity_50"}
<class 'str'>


In [78]:
b = train["amenities"]
train["amenities"] = [x.split(",") for x in train["amenities"]]

In [80]:
print(train["amenities"])

0        [{"Wireless Internet", "Air conditioning", Kit...
1        [{"Wireless Internet", "Air conditioning", Kit...
2        [{TV, "Cable TV", "Wireless Internet", "Air co...
3        [{TV, "Cable TV", Internet, "Wireless Internet...
4        [{TV, Internet, "Wireless Internet", "Air cond...
5        [{TV, "Wireless Internet", Heating, "Smoke det...
6        [{TV, Internet, "Wireless Internet", "Air cond...
7        [{TV, "Cable TV", "Wireless Internet", "Wheelc...
8        [{TV, "Cable TV", "Wireless Internet", "Pets l...
9        [{"Wireless Internet", "Air conditioning", Kit...
10       [{Internet, "Wireless Internet", "Air conditio...
11       [{TV, "Cable TV", Internet, "Wireless Internet...
12       [{TV, Internet, "Wireless Internet", Kitchen, ...
13       [{TV, "Cable TV", Internet, "Wireless Internet...
14       [{Kitchen, Heating, "Smoke detector", "Carbon ...
15       [{Internet, "Wireless Internet", "Air conditio...
16       [{TV, "Wireless Internet", Kitchen, "Pets allo.

In [21]:
import lda
import lda.datasets
X = lda.datasets.load_reuters()
vocab = lda.datasets.load_reuters_vocab()
titles = lda.datasets.load_reuters_titles()
